本文主要以kaggle一次二分类竞赛数据为例，深入研究改进forest

LayerDTree是LayerForest的精简版，是对Forest进一步探索的基础

- 数据地址：https://www.kaggle.com/c/porto-seguro-safe-driver-prediction
- 数据特点：
  - 非常不均衡、缺失值严重、二分类
- 包含内容：
  1. 数据获取
  2. 模型应用
  3. 结果分析

# Stage-1：获取数据

In [1]:
import os
import numpy as np
import pandas as pd
import time
import os.path as osp

In [2]:
class FeatureParser(object):
    def __init__(self, desc):
        desc = desc.strip()
        if desc == "C":
            self.f_type = "number"
        else:
            self.f_type = "categorical"
            f_names = [d.strip() for d in desc.split(",")]
            # missing value
            f_names.insert(0, "?")
            self.name2id = dict(zip(f_names, range(len(f_names))))

    def get_float(self, f_data):
        f_data = f_data.strip()
        if self.f_type == "number":
            return float(f_data)
        return float(self.name2id[f_data])

    def get_data(self, f_data):
        f_data = f_data.strip()
        if self.f_type == "number":
            return float(f_data)
        data = np.zeros(len(self.name2id), dtype=np.float32)
        data[self.name2id[f_data]] = 1
        return data

    def get_fdim(self):
        """
        get feature dimension
        """
        if self.f_type == "number":
            return 1
        return len(self.name2id)

In [3]:
train_data_path = osp.join(".\\datasetes", "adult", "adult.data")
test_data_path = osp.join(".\\datasetes", "adult", "adult.test")
feature_desc_path = osp.join(".\\datasetes", "adult", "features")
train_data_path, test_data_path, feature_desc_path

('.\\datasetes\\adult\\adult.data',
 '.\\datasetes\\adult\\adult.test',
 '.\\datasetes\\adult\\features')

In [4]:
f_parsers = []
with open(feature_desc_path) as f:
    for row in f.readlines():
        f_parsers.append(FeatureParser(row))
# f_parsers

In [5]:
with open(train_data_path) as f:
    rows = [row.strip().split(",") for row in f.readlines() if len(row.strip()) > 0 and not row.startswith("|")]
n_datas = len(rows)

cate_as_onehot = 0
if cate_as_onehot:
    X_dim = np.sum([f_parser.get_fdim() for f_parser in f_parsers])
    X = np.zeros((n_datas, X_dim), dtype=np.float32)
else:
    X = np.zeros((n_datas, 14), dtype=np.float32)
y = np.zeros(n_datas, dtype=np.int32)
for i, row in enumerate(rows):
    assert len(row) == 15, "len(row) wrong, i={}".format(i)
    foffset = 0
    for j in range(14):
        if cate_as_onehot:
            fdim = f_parsers[j].get_fdim()
            X[i, foffset:foffset+fdim] = f_parsers[j].get_data(row[j].strip())
            foffset += fdim
        else:
            X[i, j] = f_parsers[j].get_float(row[j].strip())
    y[i] = 0 if row[-1].strip().startswith("<=50K") else 1
print(X.shape, y.shape)
X_train = X
y_train = y

(32561, 14) (32561,)


In [6]:
with open(test_data_path) as f:
    rows = [row.strip().split(",") for row in f.readlines() if len(row.strip()) > 0 and not row.startswith("|")]
n_datas = len(rows)

cate_as_onehot = 0
if cate_as_onehot:
    X_dim = np.sum([f_parser.get_fdim() for f_parser in f_parsers])
    X = np.zeros((n_datas, X_dim), dtype=np.float32)
else:
    X = np.zeros((n_datas, 14), dtype=np.float32)
y = np.zeros(n_datas, dtype=np.int32)
for i, row in enumerate(rows):
    assert len(row) == 15, "len(row) wrong, i={}".format(i)
    foffset = 0
    for j in range(14):
        if cate_as_onehot:
            fdim = f_parsers[j].get_fdim()
            X[i, foffset:foffset+fdim] = f_parsers[j].get_data(row[j].strip())
            foffset += fdim
        else:
            X[i, j] = f_parsers[j].get_float(row[j].strip())
    y[i] = 0 if row[-1].strip().startswith("<=50K") else 1
print(X.shape, y.shape)
X_sub = X
y_sub = y

(16281, 14) (16281,)


# Stage-2：模型应用

In [7]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedShuffleSplit
import ForestUtils
import time
import random
from sklearn import metrics

In [9]:
import EnhancedDTree
import EnhancedForest
import importlib

In [10]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [11]:
importlib.reload(EnhancedForest)

<module 'EnhancedForest' from 'C:\\github_workspace\\LayerForest\\EnhancedForest.py'>

# 决策树算法

In [89]:
clf = DecisionTreeClassifier()
clf

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [90]:
clf = clf.fit(X_train, y_train)

In [93]:
p_train = clf.predict_proba(X_sub)
p_train = [item[1] for item in p_train]
p_train = np.array(p_train)
print("data auc", metrics.roc_auc_score(y_sub, p_train))

data auc 0.74431797296


In [94]:
p_train = clf.predict(X_sub)
print("data auc", metrics.accuracy_score(y_sub, p_train))

data auc 0.813402125177


# 随机森林算法

In [102]:
rf = RandomForestClassifier(n_estimators=50, max_depth=10, n_jobs=4, random_state=1024, verbose=True)
rf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=4,
            oob_score=False, random_state=1024, verbose=True,
            warm_start=False)

In [103]:
rf.fit(X_train, y_train)

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.1s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=4,
            oob_score=False, random_state=1024, verbose=True,
            warm_start=False)

In [104]:
p_train = rf.predict_proba(X_sub)
p_train = [item[1] for item in p_train]
p_train = np.array(p_train)
print("data auc", metrics.roc_auc_score(y_sub, p_train))

data auc 0.913601941746


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.0s finished


In [105]:
p_train = rf.predict(X_sub)
print("data acc", metrics.accuracy_score(y_sub, p_train))

data acc 0.859652355506


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.0s finished


# XGB算法

In [96]:
import xgboost as xgb

In [97]:
params = {
    'objective': 'binary:logistic',
    'silent': True,
    
    'max_depth': 4,
    'eta': 0.020,
    'gamma': 0.65,
    
    'colsample_bytree': 0.8,
    'subsample': 0.6,
    
    'num_boost_round' : 700,
#     'min_child_weight': 10.0,
#     'max_delta_step': 1.8,
}

In [109]:
# Define the gini metric - from https://www.kaggle.com/c/ClaimPredictionChallenge/discussion/703#5897
def ginic(actual, pred):
    actual = np.asarray(actual) #In case, someone passes Series or list
    n = len(actual)
    a_s = actual[np.argsort(pred)]
    a_c = a_s.cumsum()
    giniSum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
    return giniSum / n

def gini_normalized(a, p):
#     if p.ndim == 2:#Required for sklearn wrapper
#         p = p[:,1] #If proba array contains proba for both 0 and 1 classes, just pick class 1
    return ginic(a, p) / ginic(a, a)

# Create an XGBoost-compatible metric from Gini
def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = gini_normalized(labels, preds)
    return 'gini', gini_score

In [111]:
d_train = xgb.DMatrix(X_train, y_train)
d_valid = xgb.DMatrix(X_sub, y_sub)
watchlist = [(d_train, 'train'), (d_valid, 'valid')]

mdl = xgb.train(params, d_train, 
                    num_boost_round=1600, evals=watchlist, early_stopping_rounds=100, 
                    feval=gini_xgb, maximize=True, verbose_eval=100)

[0]	train-gini:0.691894	valid-gini:0.700265
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[100]	train-gini:0.819327	valid-gini:0.816554
[200]	train-gini:0.836354	valid-gini:0.830442
[300]	train-gini:0.847753	valid-gini:0.839445
[400]	train-gini:0.856274	valid-gini:0.845215
[500]	train-gini:0.861858	valid-gini:0.848523
[600]	train-gini:0.866693	valid-gini:0.850783
[700]	train-gini:0.870262	valid-gini:0.851864
[800]	train-gini:0.87336	valid-gini:0.853076
[900]	train-gini:0.876152	valid-gini:0.853475
[1000]	train-gini:0.878787	valid-gini:0.854279
[1100]	train-gini:0.881256	valid-gini:0.854657
[1200]	train-gini:0.883566	valid-gini:0.854888
[1300]	train-gini:0.885647	valid-gini:0.855086
[1400]	train-gini:0.887623	valid-gini:0.855218
[1500]	train-gini:0.88951	valid-gini:0.85506
Stopping. Best iteration:
[1412]	train-gini:0.88786	valid-gini:0.855257



In [113]:
d_test = xgb.DMatrix(X_sub)
p_train = mdl.predict(d_test)
print("data auc", metrics.roc_auc_score(y_sub, p_train))

data auc 0.927533951378


In [114]:
test_y_acc_index = np.where(p_train > 0.5)[0]
test_y_acc = np.array([0] * len(p_train))
test_y_acc[test_y_acc_index] = 1
metrics.accuracy_score(y_sub, test_y_acc)

0.87414777962041645

# layerDTree算法

In [99]:
importlib.reload(EnhancedForest)

<module 'EnhancedForest' from 'C:\\github_workspace\\LayerForest\\EnhancedForest.py'>

In [72]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)

def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

def gini_metrix(a, p):
    return "nor gini:", gini_normalized(a,p)

In [108]:
X = X_train.copy()
y = y_train.copy()

X_test = X_sub.copy()
test_y = np.array(([0.0] * len(X_test)))
all_data_mask = np.array([False] * len(X_test))
real_y = y_sub.copy()

# 均衡数据进行layer
X_train_np = X
y_train_np = y
maxlayer = 300
layer = 0

enhancedForest = EnhancedForest.EnhancedForest()
counter = 0
while 1:
    layer += 1
    X = X_train_np
    y = y_train_np
    
    # 均衡数据
    positive_mask = np.where(y == 1)[0]
    negative_index = np.where(y == 0)[0]
    random.shuffle(negative_index)
    negative_mask = negative_index[:len(positive_mask)]
    train_mask = np.hstack((positive_mask, negative_mask))
    train_data_x = X[train_mask]
    train_data_y = y[train_mask]
    guest_mask = negative_index[len(positive_mask):]
    guest_data_x = X[guest_mask]
    guest_data_y = y[guest_mask]
    
#     print("train==1", train_data_y[train_data_y == 1].shape)
#     print("train==0", train_data_y[train_data_y == 0].shape)
    clf, data_mask, all_false_data_index, p_test  = \
        enhancedForest.TrainModelLayer(train_data_x, train_data_y, X_test, all_data_mask, test_y, real_y, verbose=False, feval=gini_metrix)
        
    X_train_np = enhancedForest.X_train_np
    y_train_np = enhancedForest.y_train_np
    
    # 均衡数据
    X_train_np = np.vstack((X_train_np, guest_data_x))
    y_train_np = np.hstack((y_train_np, guest_data_y))
#     print("train==1", y_train_np[y_train_np == 1].shape)
#     print("train==0", y_train_np[y_train_np == 0].shape)
    pass_data_id = data_mask[data_mask==True]
    all_false_data_index = np.where(all_data_mask == False)[0]
    X_test_np = X_test[all_false_data_index]
    print("%d [%d/%d] " % (layer, len(pass_data_id), len(X_test_np) - len(pass_data_id)), end="")
    
    if X_train_np.shape[0] < 1000 or y_train_np[y_train_np==1].shape[0] <= 10 \
        or layer > maxlayer \
        or len(p_test[~data_mask]) == 0:
        all_data_mask = all_false_data_index[~data_mask]
        test_y[all_data_mask] = p_test[~data_mask]
        print(len(p_test[~data_mask]))
        break
        
    all_data_mask[~all_data_mask] = data_mask

[812/14870] 1 [1261/15020] [912/14586] 2 [656/14364] [400/14886] 3 [224/14140] [327/14901] 4 [145/13995] [320/14818] 5 [131/13864] [322/14624] 6 [90/13774] [283/14587] 7 [100/13674] [147/14673] 8 [64/13610] [107/14711] 9 [39/13571] [235/14547] 10 [64/13507] [75/14625] 11 [2/13505] [147/14551] 12 [82/13423] [153/14439] 13 [56/13367] [139/14437] 14 [44/13323] [215/14309] 15 [115/13208] [110/14406] 16 [42/13166] [100/14412] 17 [41/13125] [77/14431] 18 [34/13091] [63/14399] 19 [14/13077] [103/14351] 20 [64/13013] [61/14391] 21 [15/12998] [200/14242] 22 [121/12877] [38/14392] 23 [15/12862] [74/14348] 24 [24/12838] [83/14327] 25 [10/12828] [87/14323] 26 [40/12788] [34/14310] 27 [17/12771] [99/14235] 28 [38/12733] [66/14174] 29 [16/12717] [24/14200] 30 [21/12696] [42/14174] 31 [16/12680] [22/14182] 32 [7/12673] [85/14117] 33 [60/12613] [17/14179] 34 [5/12608] [12/14164] 35 [4/12604] [37/14129] 36 [13/12591] [25/14131] 37 [12/12579] [109/14037] 38 [35/12544] [26/14056] 39 [10/12534] [29/14051]

In [160]:
# 均衡数据进行layer

In [109]:
metrics.roc_auc_score(y_sub, test_y)

0.90604666888726215

In [110]:
test_y_acc_index = np.where(test_y > 0.5)[0]
test_y_acc = np.array([0] * len(test_y))
test_y_acc[test_y_acc_index] = 1
metrics.accuracy_score(y_sub, test_y_acc)

0.81635034703028064

In [ ]:
# 不均衡数据进行layer

In [101]:
metrics.roc_auc_score(y_sub, test_y)

0.90945992483848936

In [81]:
test_y_acc_index = np.where(test_y > 0.5)[0]
test_y_acc = np.array([0] * len(test_y))
test_y_acc[test_y_acc_index] = 1
metrics.accuracy_score(y_sub, test_y_acc)

0.85430870339659726

In [100]:
X = X_train.copy()
y = y_train.copy()

X_test = X_sub.copy()
test_y = np.array(([0.0] * len(X_test)))
all_data_mask = np.array([False] * len(X_test))
real_y = y_sub.copy()

# 不均衡数据进行layer
X_train_np = X
y_train_np = y
maxlayer = 300
layer = 0

enhancedDTree = EnhancedForest.EnhancedForest()
counter = 0
while 1:
    layer += 1
#     print()
#     print("layer:", layer)
    X = X_train_np
    y = y_train_np
    clf, data_mask, all_false_data_index, p_test  = \
        enhancedDTree.TrainModelLayer(X, y, X_test, all_data_mask, test_y, real_y, verbose=False, feval=gini_metrix)
    X_train_np = enhancedDTree.X_train_np
    y_train_np = enhancedDTree.y_train_np
    
    # 打印信息
    pass_data_id = data_mask[data_mask==True]
    all_false_data_index = np.where(all_data_mask == False)[0]
    X_test_np = X_test[all_false_data_index]
    print("%d [%d/%d] " % (layer, len(pass_data_id), len(X_test_np) - len(pass_data_id)), end="")
    
    if X_train_np.shape[0] < 1000 or layer > maxlayer or y_train_np[y_train_np==1].shape[0] <= 10:
        all_data_mask = all_false_data_index[~data_mask]
        test_y[all_data_mask] = p_test[~data_mask]
        print(len(p_test[~data_mask]))
        break
        
    all_data_mask[~all_data_mask] = data_mask

[7144/25417] 1 [3517/12764] [1440/23977] 2 [766/11998] [731/23246] 3 [372/11626] [306/22940] 4 [163/11463] [223/22717] 5 [107/11356] [222/22495] 6 [100/11256] [154/22341] 7 [83/11173] [229/22112] 8 [102/11071] [137/21975] 9 [68/11003] [89/21886] 10 [44/10959] [165/21721] 11 [84/10875] [68/21653] 12 [33/10842] [41/21612] 13 [24/10818] [24/21588] 14 [15/10803] [57/21531] 15 [29/10774] [28/21503] 16 [21/10753] [19/21484] 17 [8/10745] [52/21432] 18 [30/10715] [34/21398] 19 [13/10702] [59/21339] 20 [30/10672] [22/21317] 21 [6/10666] [18/21299] 22 [7/10659] [33/21266] 23 [14/10645] [55/21211] 24 [22/10623] [8/21203] 25 [3/10620] [5/21198] 26 [3/10617] [16/21182] 27 [6/10611] [64/21118] 28 [32/10579] [12/21106] 29 [8/10571] [7/21099] 30 [7/10564] [25/21074] 31 [15/10549] [12/21062] 32 [4/10545] [16/21046] 33 [8/10537] [17/21029] 34 [8/10529] [60/20969] 35 [31/10498] [15/20954] 36 [10/10488] [14/20940] 37 [9/10479] [16/20924] 38 [8/10471] [35/20889] 39 [8/10463] [11/20878] 40 [8/10455] [13/208

### Todo list
- 树结构设计（完成）
- 通过gini对数据分割（完成）
- 全局测试集
- 输出结果集
- 打印信息增加pass data的比例
- 防止过拟合
- 对pass data的进一步处理
- 先进行数据均衡化是不是更快一些